In [1]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [2]:
!pip install transformers
!pip install accelerate
!pip install huggingface_hub
!pip install flask
!pip install flask-ngrok
!pip install pyngrok==4.1.1
!pip install -U flask-cors

   ---------------------------------------- 0.0/314.1 kB ? eta -:--:--
   ---------------------------------------- 0.0/314.1 kB ? eta -:--:--
   - -------------------------------------- 10.2/314.1 kB ? eta -:--:--
   ----- --------------------------------- 41.0/314.1 kB 393.8 kB/s eta 0:00:01
   ---------- ---------------------------- 81.9/314.1 kB 512.0 kB/s eta 0:00:01
   ----------- --------------------------- 92.2/314.1 kB 476.3 kB/s eta 0:00:01
   ------------- ------------------------ 112.6/314.1 kB 437.6 kB/s eta 0:00:01
   ------------------- ------------------ 163.8/314.1 kB 579.6 kB/s eta 0:00:01
   --------------------- ---------------- 174.1/314.1 kB 525.1 kB/s eta 0:00:01
   --------------------------- ---------- 225.3/314.1 kB 625.1 kB/s eta 0:00:01
   --------------------------------- ---- 276.5/314.1 kB 655.8 kB/s eta 0:00:01
   --------------------------------- ---- 276.5/314.1 kB 655.8 kB/s eta 0:00:01
   --------------------------------- ---- 276.5/314.1 kB 655.8 kB/

In [3]:
!huggingface-cli login

^C


In [ ]:
from transformers import AutoTokenizer
import transformers
import torch

model = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Summarizer

In [ ]:
def summarize(prompt):
  summarizer_instruction = f"""
    <s>[INST]<<SYS>>
      You are a summarizer and your job is to summarize whatever user asks you too in a most concise manner while being safe. Your summaries must have the same meaning and not include and false information. Make sure you dont use any external knowledge other than whats provided to you. Your final output must only be the summarize text.
    <</SYS>>
    {prompt}
    [/INST]
  """
  sequences = pipeline(
      summarizer_instruction,
      do_sample=True,
      top_k=10,
      num_return_sequences=1,
      eos_token_id=tokenizer.eos_token_id,
  )

  answer = sequences[0]['generated_text'].split("[/INST]")[1].strip()
  return answer

In [ ]:
prompt = """Hodgkin left Benwell Dene in 1887, aged ten, to spend the first of four years at the preparatory Seabank School, in Alnmouth, though several months of his first year were spent at home recovering from bronchitis.By all accounts, Hodgkin's first years at school were unhappy. "School at Alnmouth was not a bit a happy time", one sister later recalled; "when he was old he used to say that it did seem strange that the hardest lessons you had to learn came at the very beginning of life."[14] Another noted that "he hated it too much to talk about it", and that when older, he stated "The one thing to be said for it was that all later miseries seemed nothing by comparison".Hodgkin's wife, meanwhile, termed him a "home lover" for whom the time at Seabank was "more or less unhappy".Hodgkin would pray for the school to burn down, and one year his prayers were answered, in a fashion, when an outbreak of scarlet fever led to all the boys being sent home.The situation improved slightly when Hodgkin\'s brother George, three and a half years his junior, entered Seabank, and (as his sister wrote), "Robin became resigned"""
summarize(prompt)

In [ ]:
!curl -s https://ngrok-agent.s3.amazonaws.com/ngrok.asc | sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null && echo "deb https://ngrok-agent.s3.amazonaws.com buster main" | sudo tee /etc/apt/sources.list.d/ngrok.list && sudo apt update && sudo apt install ngrok

In [ ]:
!ngrok authtoken <YOUR_NGROK_AUTH_TOKEN_HERE>

In [ ]:
from flask import Flask, request
from flask_ngrok import run_with_ngrok
from flask_cors import CORS, cross_origin

app = Flask(__name__)
cors = CORS(app)
app.config['CORS_HEADERS'] = 'Content-Type'

run_with_ngrok(app)

@app.route("/summarize")
@cross_origin()

def home():
  summarizing_text = request.args.get('text')
  if len(summarizing_text) == 0:
    return {
        "message": "Text is empty",
        "code": 400
    }, 400

  text = summarize(summarizing_text)
  return {
      "summarized_text": text,
      "code": 200
  },200
app.run()